In [25]:
import os 
import google.auth

from google.oauth2 import service_account

from config import config_env

from function.function import  FromDriveToStorage
import google.auth


CONFIG_ENV = os.getenv("ENV") or "dev"
config = config_env[CONFIG_ENV]

# Create credentials with Drive & BigQuery API scopes.
# Both APIs must be enabled for your project before running this code.
# credentials, project = google.auth.default(
#     scopes=config.SCOPES
# )

key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/key_storage.json'
creds=None
if CONFIG_ENV=="dev":
     creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    
     scoped_credentials = creds.with_scopes(config.SCOPES)    
else:
    creds, _ = google.auth.default()
    scoped_credentials = creds.with_scopes(config.SCOPES)


FromDriveToStorage(config, creds=creds, scoped_credentials=scoped_credentials)


In [44]:
import google.oauth2.credentials

key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/kk.json'
credentials = google.oauth2.credentials.Credentials(key_path)

In [45]:
credentials._refresh_token


In [53]:
credentials = google.oauth2.credentials.Credentials(
    "d4PMufmHKVwcPc1IY40n0UnduTfjTd",
    refresh_token=None,
    token_uri="https://oauth2.googleapis.com/token",
    client_id='318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com',
    client_secret='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')
credentials

In [70]:
import google.oauth2.credentials
import google_auth_oauthlib.flow

# Use the client_secret.json file to identify the application requesting
# authorization. The client ID (from that file) and access scopes are required.
key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/kk.json'

flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/drive.metadata.readonly'])

# Indicate where the API server will redirect the user after the user completes
# the authorization flow. The redirect URI is required. The value must exactly
# match one of the authorized redirect URIs for the OAuth 2.0 client, which you
# configured in the API Console. If this value doesn't match an authorized URI,
# you will get a 'redirect_uri_mismatch' error.
flow.redirect_uri = 'https://www.example.com/oauth2callback'

# Generate URL for request to Google's OAuth 2.0 server.
# Use kwargs to set optional request parameters.
authorization_url, state = flow.authorization_url(
    # Enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true')

In [76]:
from googleapiclient.discovery import build

from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_secrets_file(
                key_path, config.SCOPES)
creds = flow.run_local_server(port=0)
service=build('drive', 'v3', credentials=creds)
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50773%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly&state=ujSNj34HYooyY32DEJt0rN1HAfUPZQ&access_type=offline


In [84]:

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
SAMPLE_RANGE_NAME = 'Class Data!A2:E'


def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)
        results = service.files().list(
            pageSize=1, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
        # get the results
        items = results.get('files', [])
        # list all 20 files & folders
        print(items)

    except HttpError as err:
        print(err)
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A34619%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=XcXfyno1ZLhyZpkaNFpQFEiJeidcax&access_type=offline


KeyboardInterrupt: 

In [82]:
from oauth2client import client, file, tools

import pickle
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']


def get_gdrive_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            name ="a"
            storage = file.Storage(name + ".dat")
            creds = tools.run_flow(flow, storage)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    # return Google Drive API service
    return build('drive', 'v3', credentials=creds)




"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 5 files the user has access to.
"""
service = get_gdrive_service()
# Call the Drive v3 API
results = service.files().list(
    pageSize=1, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
# get the results
items = results.get('files', [])
# list all 20 files & folders
items


[{'mimeType': 'text/csv',
  'parents': ['0AE_1lvoLHZoqUk9PVA'],
  'size': '32036',
  'id': '1AlS6e0U9cwnEpN7vLpXV_XUVkL6boFhR',
  'name': 'Copia de ABA.csv',
  'modifiedTime': '2023-02-28T09:26:39.260Z'}]

In [68]:

import argparse
import os

from googleapiclient import discovery
from googleapiclient.http import build_http
def init(
     name, version,  scope=None, parents=[], discovery_filename=None
):
    """A common initialization routine for samples.
    Many of the sample applications do the same initialization, which has now
    been consolidated into this function. This function uses common idioms found
    in almost all the samples, i.e. for an API with name 'apiname', the
    credentials are stored in a file named apiname.dat, and the
    client_secrets.json file is stored in the same directory as the application
    main file.
    Args:
      argv: list of string, the command-line parameters of the application.
      name: string, name of the API.
      version: string, version of the API.
      doc: string, description of the application. Usually set to __doc__.
      file: string, filename of the application. Usually set to __file__.
      parents: list of argparse.ArgumentParser, additional command-line flags.
      scope: string, The OAuth scope used.
      discovery_filename: string, name of local discovery file (JSON). Use when discovery doc not available via URL.
    Returns:
      A tuple of (service, flags), where service is the service object and flags
      is the parsed command-line flags.
    """
    try:
        from oauth2client import client, file, tools
    except ImportError:
        raise ImportError(
            "googleapiclient.sample_tools requires oauth2client. Please install oauth2client and try again."
        )

    if scope is None:
        scope = "https://www.googleapis.com/auth/" + name

    # Parser command-line arguments.
    parent_parsers = [tools.argparser]
    parent_parsers.extend(parents)
    
    # flags = parser.parse_args(argv[1:])

    # Name of a file containing the OAuth 2.0 information for this
    # application, including client_id and client_secret, which are found
    # on the API Access tab on the Google APIs
    # Console <http://code.google.com/apis/console>.
    client_secrets = key_path

    # Set up a Flow object to be used if we need to authenticate.
    flow = client.flow_from_clientsecrets(
        client_secrets, scope=scope, message=tools.message_if_missing(client_secrets)
    )

    # Prepare credentials, and authorize HTTP object with them.
    # If the credentials don't exist or are invalid run through the native client
    # flow. The Storage object will ensure that if successful the good
    # credentials will get written back to a file.
    http = credentials.authorize(http=build_http())

    if discovery_filename is None:
        # Construct a service object via the discovery service.
        service = discovery.build(name, version, http=http)
    else:
        # Construct a service object using a local discovery document file.
        with open(discovery_filename) as discovery_file:
            service = discovery.build_from_document(
                discovery_file.read(), base="https://www.googleapis.com/", http=http
            )
    return (service, _)

In [69]:
service, flags = init(
        "drive",
        "v3",                
        scope="https://www.googleapis.com/auth/drive.readonly",
    )


AttributeError: 'Credentials' object has no attribute 'authorize'

In [26]:
FromDriveToStorage.run()

[]


In [55]:
from googleapiclient.discovery import build
service = build('drive', 'v3', credentials=credentials)

In [56]:
folder_ids ="1uymD6k_4JTTP4YuTP_NBTAftS4nteU5o"
service.files().list(q=f"'{folder_ids}' in parents").execute()

RefreshError: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.

In [34]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

credentials = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    
scoped_credentials = credentials.with_scopes(config.SCOPES)    



try:
    service = build("drive", "v3", credentials=scoped_credentials)
    files = []
    page_token = None
    while True:
        response = service.files().list(
            spaces="drive",
            fields="nextPageToken, files(id, name)",
            pageToken=page_token).execute()
        for file in response.get("files", []):
            print(f"Found file: {file.get('name')}, {file.get('id')}")
        files.extend(response.get("files", []))
        page_token = response.get("nextPageToken", None)
        if page_token is None:
            break
except HttpError as error:
    print(f"An error occurred: {error}")
    files = None

MalformedError: Service account info was not in the expected format, missing fields client_email.

In [30]:
files

[]

In [39]:
import google.oauth2.credentials

credentials = google.oauth2.credentials.Credentials(key_path)
service = build('drive', 'v3', credentials=credentials)
# Llama al API para listar los archivos
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])


RefreshError: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.

In [38]:
service

In [33]:
import os
import google.auth
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Path al archivo de credenciales
key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/kk.json'
credential_path = key_path

# Carga las credenciales desde el archivo
creds = None
if os.path.exists(credential_path):
    creds = Credentials.from_service_account_file(credential_path)

# Construye el cliente de la API de Google Drive
service = build('drive', 'v3', credentials=creds)


AttributeError: type object 'Credentials' has no attribute 'from_service_account_file'

In [ ]:
# Llama al API para listar los archivos
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

# Imprime el resultado
if not items:
    print('No se encontraron archivos.')
else:
    print('Archivos:')
    for item in items:
        print(f'{item["name"]} ({item["id"]})')
